# **1. Chargement et Transformation des Données**

In [82]:
import warnings
warnings.filterwarnings('ignore')

import chess

import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

## A. Chargement des Données

In [83]:
data = pd.read_csv('data/data_cleaned.csv' )

X = data['fen']
y = data['most_popular_move_of_the_category']

## B. Transformation des Données

Le reseaux de neurone ne peut pas interpreter directement la position sous forme de FEN ni le coup d'échec sous la forme UCI :

- **Position (Fen)** -> Matrice Numpy
- **Coup a joué (UCI)** -> Encodage de chaque coup unique (sous forme d'entier)

In [84]:
def fen_to_matrix(fen: str):
    """
    Utilité: 
        Convertit une position d'échecs en FEN en une matrice 8x8x12.
        8*8 pour les cases de l'échequier et *12 pour chaque pieces uniques (6 Blanches et 6 Noires)
    
    Parameters: 
        La représentation FEN de la position.
    
    Returns:
        Une matrice numpy 8x8x12 représentant la position.
    """
    board = chess.Board(fen) #  convertit une représentation FEN en un objet manipulable qui contient toutes les informations sur la position d'échecs. Cet objet permet ensuite de travailler directement avec l'échiquier dans le code.
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()

    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1  # Type de pièce (0 pour pion, 1 pour cavalier, etc.)
        piece_color = 0 if piece.color == chess.WHITE else 6  # 0-5 pour blanc, 6-11 pour noir
        matrix[row, col, piece_type + piece_color] = 1

    return matrix

In [85]:
def encode_moves(moves):
    """
    Utilité :
        Cette fonction prend une liste de coups d'échecs (sous forme de chaînes) et encode chaque coup unique
        sous forme d'un entier unique. Elle retourne également un dictionnaire qui associe chaque coup à son
        entier correspondant.

    Parameters:
        moves (list of str): A list of chess moves as strings (e.g., ['e2e4', 'd2d4']).

    Returns:
        tuple:
            - A list of integers representing the encoded moves (e.g., [0, 1, 0]).
            - A dictionary mapping each unique move to its corresponding integer (e.g., {'e2e4': 0, 'd2d4': 1}).

    """
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [86]:
X = X.apply(fen_to_matrix)
X = np.array(X.tolist())

y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))

# **2. Entrainement du Modèle**

## **Model 1** :

1. **`Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12))`** : 
   - C'est une couche de convolution 2D qui extrait des caractéristiques spatiales de l'échiquier. 
   - **64 filtres** sont utilisés pour capter des motifs complexes, avec une taille de filtre de **3x3**, idéale pour des images petites comme un échiquier **8x8**. 
   - La fonction d'activation **ReLU** accélère l'apprentissage en gérant les gradients de manière efficace.

2. **`Conv2D(128, (3, 3), activation='relu')`** : 
   - Une deuxième couche de convolution avec **128 filtres** permet de capturer des motifs plus complexes et abstraits sur l'échiquier.
   - La taille des filtres reste de **3x3**, et **ReLU** est utilisé pour améliorer l'apprentissage.

3. **`Flatten()`** : 
   - Cette couche aplatit les sorties **2D** des couches précédentes en un vecteur **1D**, ce qui est nécessaire pour connecter les couches de convolution aux couches entièrement connectées.

4. **`Dense(256, activation='relu')`** : 
   - Une couche entièrement connectée avec **256 neurones** pour capturer des relations non-linéaires complexes entre les caractéristiques extraites par les convolutions.
   - La fonction **ReLU** est utilisée pour une meilleure gestion des non-linéarités.

5. **`Dense(len(move_to_int), activation='softmax')`** : 
   - La couche de sortie a un nombre de neurones égal au nombre de **coups possibles**. La fonction **Softmax** est utilisée pour classer les coups possibles en leur attribuant une probabilité, ce qui permet de sélectionner le coup le plus probable.

In [87]:
model_1 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])

In [91]:
model_1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [92]:
model_1.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_25 (Conv2D)              │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1908)           │       490,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,095,732 (4.18 MB)

 Trainable params: 1,095,732 (4.18 MB)

 Non-trainable params: 0 (0.00 B)

In [94]:
model_1.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 152s 21ms/step - accuracy: 0.0430 - loss: 6.0971 - val_accuracy: 0.1705 - val_loss: 3.6178
Epoch 2/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 151s 21ms/step - accuracy: 0.1092 - loss: 4.5072 - val_accuracy: 0.1889 - val_loss: 3.4085
Epoch 3/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 153s 22ms/step - accuracy: 0.1376 - loss: 4.0786 - val_accuracy: 0.1964 - val_loss: 3.3292
Epoch 4/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 135s 19ms/step - accuracy: 0.1543 - loss: 3.8623 - val_accuracy: 0.2000 - val_loss: 3.3157
Epoch 5/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 116s 16ms/step - accuracy: 0.1709 - loss: 3.7118 - val_accuracy: 0.1990 - val_loss: 3.3421
Epoch 6/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 132s 19ms/step - accuracy: 0.1833 - loss: 3.6081 - val_accuracy: 0.2031 - val_loss: 3.3164
Epoch 7/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 160s 23ms/step - accuracy: 0.1954 - loss: 3.5199 - val_accuracy: 0.2125 - val_loss: 3.3126
Epoch 8/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 173s 25ms/step - accuracy: 

Sauvegarde du Modèle

In [95]:
model_1.save("Models/Modele_1_TF_25EPOCHS.keras")

## **Model 2**

Ajout de plus de couches de convolution permet de capturer des motifs encore plus complexes. Cependant, cela peut augmenter le risque de sur-apprentissage, donc il faudra ajuster les autres hyperparamètres (comme le taux d'apprentissage ou la régularisation).

In [96]:
model_2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),  # Ajout d'une couche supplémentaire
    Flatten(),
    Dense(512, activation='relu'),  # Augmentation de la taille de la couche dense
    Dense(len(move_to_int), activation='softmax')
])

In [97]:
model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
model_2.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 2, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1908)           │       978,804 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,879,604 (7.17 MB)

 Trainable params: 1,879,604 (7.17 MB)

 Non-trainable params: 0 (0.00 B)

In [99]:
model_2.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 256s 36ms/step - accuracy: 0.0290 - loss: 6.3477 - val_accuracy: 0.1397 - val_loss: 3.9145
Epoch 2/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 227s 32ms/step - accuracy: 0.0977 - loss: 4.8137 - val_accuracy: 0.1805 - val_loss: 3.5730
Epoch 3/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 265s 38ms/step - accuracy: 0.1277 - loss: 4.3076 - val_accuracy: 0.1896 - val_loss: 3.4792
Epoch 4/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 296s 42ms/step - accuracy: 0.1476 - loss: 4.0496 - val_accuracy: 0.1874 - val_loss: 3.5130
Epoch 5/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 273s 39ms/step - accuracy: 0.1627 - loss: 3.8794 - val_accuracy: 0.1890 - val_loss: 3.4614
Epoch 6/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 1209s 172ms/step - accuracy: 0.1761 - loss: 3.7578 - val_accuracy: 0.2000 - val_loss: 3.4484
Epoch 7/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 215s 31ms/step - accuracy: 0.1909 - loss: 3.6459 - val_accuracy: 0.1984 - val_loss: 3.5212
Epoch 8/25
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 220s 31ms/step - accuracy

In [100]:
model_2.save("Models/Modele_2_TF_25EPOCHS.keras")

## **Model 3**

Moins de neurones dans la couche dense peut rendre le modèle plus rapide à entraîner et plus facile à régulariser, tout en conservant une bonne capacité de généralisation.

In [101]:
model_3 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dropout(0.5),  # Dropout avec un taux de 50%
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])

In [102]:
model_3.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [103]:
model_3.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)              │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1908)           │       490,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,095,732 (4.18 MB)

 Trainable params: 1,095,732 (4.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_3.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

In [ ]:
model_3.save("Models/Modele_3_TF_25EPOCHS.keras")